In [6]:
!python preprocess2.py --config_file exp-configs/EMEA/config-125M-nl.json

2024-06-10 14:06:16,642 - INFO - Parsing arguments...
Parsing arguments...
2024-06-10 14:06:16,642 - INFO - Loading tokenizer...
Loading tokenizer...
2024-06-10 14:06:17,006 - INFO - ==== Starting data preprocessing script ====
==== Starting data preprocessing script ====
2024-06-10 14:06:17,006 - INFO - This may take a while depending on the size of the dataset...
This may take a while depending on the size of the dataset...
2024-06-10 14:06:17,006 - INFO - Counting tokens for en...
Counting tokens for en...
Generating byte offset dataset from file:  EMEA/EMEA.en
2024-06-10 14:06:36,114 - INFO - Number of samples >= 150 tokens in ./datasets/EMEA/csv/150/EMEA.en.csv: 840
Number of samples >= 150 tokens in ./datasets/EMEA/csv/150/EMEA.en.csv: 840
2024-06-10 14:06:36,115 - INFO - Counting tokens for nl...
Counting tokens for nl...
Generating byte offset dataset from file:  EMEA/EMEA.nl
2024-06-10 14:07:02,415 - INFO - Number of samples >= 150 tokens in ./datasets/EMEA/csv/150/EMEA.nl.csv

In [4]:
!python process_data.py --config_file exp-configs/EMEA/config-125M-nl.json

2024-06-10 12:47:15,808 - INFO - Parsing arguments...
Parsing arguments...
2024-06-10 12:47:15,809 - INFO - Loading tokenizer...
Loading tokenizer...
2024-06-10 12:47:16,097 - INFO - ==== Sarting data processing script ====
==== Sarting data processing script ====
2024-06-10 12:47:16,097 - INFO - This may take a while depending on the size of the dataset...
This may take a while depending on the size of the dataset...
2024-06-10 12:47:16,097 - INFO - Counting tokens for en...
Counting tokens for en...
Generating byte offset dataset from file:  EMEA/EMEA.en
2024-06-10 12:47:35,185 - INFO - Number of samples >= 150 tokens in ./datasets/EMEA/csv/150/EMEA.en.csv: 840
Number of samples >= 150 tokens in ./datasets/EMEA/csv/150/EMEA.en.csv: 840
2024-06-10 12:47:35,185 - INFO - Filtering sentences for en...
Filtering sentences for en...
2024-06-10 12:47:35,451 - INFO - Generating JSONL for en...
Generating JSONL for en...
2024-06-10 12:47:37,412 - INFO - Counting tokens for nl...
Counting toke

In [1]:
from data_lib import filter_large_entries

In [2]:
filter_large_entries("datasets/EMEA/csv/150/EMEA.en.csv", "datasets/EMEA/csv/150/EMEA.en.jsonl", 150)